<a href="https://colab.research.google.com/github/aaronhowellai/machine-learning-projects/blob/main/Introduction_to_PyTorch_Tensors%2C_MLPs%2C_DataLoading%2C_Matmul%2C_Training%2C_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction PyTorch** (Exploring the Basics)
Exploring the basics of PyTorch syntax, tensors, matrix multiplication, and backpropagation with Appendix A from the book Build a Large Language Model (From Scratch) by Sebastian Raschka.

In [ ]:
# importing packages
import torch
import torch.nn.functional as F
from torch.autograd import grad
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
torch.__version__

'2.6.0+cu124'

In [ ]:
# is optimization for gpus available?
torch.cuda.is_available()

True

# **Basics of Tensor Operations**
![image description](https://raw.githubusercontent.com/aaronhowellai/machine-learning-projects/4ca381a4e46d5f6c18bd4cd812c1513c9b99fa70/ranks%20of%20tensors.png)



In [ ]:
# scalar 0D tensor
tensor0 = torch.tensor(1)
tensor0

tensor(1)

In [ ]:
# vector 1D tensor
tensor1 = torch.tensor([1,2,3])
tensor1

tensor([1, 2, 3])

In [ ]:
# matrix 2D tensor
tensor2 = torch.tensor([
    [1,2,3],
    [4,5,6]
    ])
tensor2

tensor([[1, 2, 3],
        [4, 5, 6]])

In [ ]:
# matrix 3D tensor
tensor3 = torch.tensor([
    [[1,2,3],
    [4,5,6]],
    [[5,6,7],
     [8,9,10]]
    ])
tensor3

tensor([[[ 1,  2,  3],
         [ 4,  5,  6]],

        [[ 5,  6,  7],
         [ 8,  9, 10]]])

In [ ]:
# check the shape of the 3D tensor
tensor3.shape

torch.Size([2, 2, 3])

In [ ]:
# matrix higher order tensor
tensor4 = torch.tensor([
    [[1,2,3],
    [4,5,6]],
    [[5,6,7],
    [8,9,10]],
    [[11,12,13],
     [14,15,16]]
])

# check the shape
tensor4.shape

torch.Size([3, 2, 3])

In [ ]:
# check the type of the 1D tensor
type(tensor1)

torch.Tensor

In [ ]:
# (3,2) tensor
ex2d = torch.tensor([[1,2],[3,4],[5,6]])
ex2d

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [ ]:
# transpose of previous tensor, now (2,3)
print(ex2d.T)

tensor([[1, 3, 5],
        [2, 4, 6]])


In [ ]:
# matrix multiplication between the matrix and its transposed counterpart
print(ex2d.matmul(ex2d.T))

tensor([[ 5, 11, 17],
        [11, 25, 39],
        [17, 39, 61]])


In [ ]:
# another 2D tensor
tensor2d = torch.tensor([[1,2,3],
                         [4,5,6]])
tensor2d

tensor([[1, 2, 3],
        [4, 5, 6]])

In [ ]:
# computing the transpose as a variable
tensor2dT = tensor2d.T
tensor2dT

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [ ]:
# matmul
print(tensor2d.matmul(tensor2dT))

tensor([[14, 32],
        [32, 77]])


In [ ]:
# another way to compute matrix multiplcation
print(tensor2d @ tensor2dT)

tensor([[14, 32],
        [32, 77]])


# **Exploring Simple Neural Networks with FFNs and MLPs**

In [ ]:
# feed forward network setup for logistic regression

y = torch.tensor([1.0]) # true label
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2], requires_grad=True) # weight parameter
b = torch.tensor([0.0], requires_grad=True) # bias input

# net input
z = x1 * w1 + b

# sigmoid activation function
a = torch.sigmoid(z)

# output
loss = F.binary_cross_entropy(a,y)

# computing gradients
grad_L_w1 = grad(loss, w1, retain_graph=True) # keep computation graph
grad_L_b = grad(loss, b, retain_graph=True)

In [ ]:
# compute the values of the loss
print('Loss function of w1:',grad_L_w1)
print('Loss function of b:',grad_L_b)

Loss function of w1: (tensor([-0.0898]),)
Loss function of b: (tensor([-0.0817]),)


In [ ]:
# backward on loss as an attribute
loss.backward()

In [ ]:
# automatic results
print('Loss function of w1:',w1.grad)
print('Loss function of b:',b.grad)

Loss function of w1: tensor([-0.0898])
Loss function of b: tensor([-0.0817])


In [ ]:
# multilayer perceptron with two hidden layers
class NeuralNetwork(torch.nn.Module):
  def __init__(self, num_inputs, num_outputs):
    super().__init__()

    self.layers = torch.nn.Sequential(

    # 1st hidden layer
    torch.nn.Linear(num_inputs, 30),
    torch.nn.ReLU(),

    # 2nd hidden layer
    torch.nn.Linear(30, 20),
    torch.nn.ReLU(),

    # output layer
    torch.nn.Linear(20, num_outputs)

    )

  def forward(self, x):
    logits = self.layers(x)
    return logits

In [ ]:
# instantiate a new neural network
model = NeuralNetwork(50, 3)

print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [ ]:
# check the number of trainable parameters in this model
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [ ]:
# access Linear layer at idx 0
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0842, -0.0894, -0.0503,  ...,  0.0909,  0.0059,  0.1195],
        [ 0.0767,  0.0866, -0.0182,  ...,  0.1335, -0.0243, -0.0421],
        [-0.0731, -0.0866, -0.0313,  ..., -0.1188, -0.0511,  0.1353],
        ...,
        [-0.1058, -0.0454,  0.0523,  ...,  0.0103, -0.1246,  0.0234],
        [ 0.1052, -0.0829,  0.0643,  ..., -0.0337,  0.0725,  0.0059],
        [ 0.0122, -0.0581, -0.0625,  ..., -0.0168, -0.0615,  0.1057]],
       requires_grad=True)


In [ ]:
# show its dimensions
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [ ]:
# access the bias vector
print(model.layers[0].bias)

Parameter containing:
tensor([-0.0386, -0.0358,  0.0491,  0.0413, -0.0890,  0.1030, -0.0516,  0.1405,
         0.1137, -0.1332, -0.0899, -0.1237,  0.1245,  0.0925, -0.0469,  0.0179,
         0.0634,  0.1312, -0.1269,  0.0375,  0.0918, -0.1243, -0.0291,  0.1386,
        -0.0277,  0.0167, -0.1267,  0.0379,  0.0358,  0.1376],
       requires_grad=True)


In [ ]:
# weight matrix random number seed
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


## `NeuralNetwork()` with Forward Pass
Below shows a single random training example X as a dummy input, fed to the model, returning 3 scores.

In [ ]:
# forward pass
torch.manual_seed(123)
X = torch.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [ ]:
# compute the class-membership probabilities for predictions

# call softmax function
with torch.no_grad():
  out = torch.softmax(model(X), dim=1)

print(out)

tensor([[0.3113, 0.3934, 0.2952]])


# **Setting up Efficient Data Loaders**
![image description](https://raw.githubusercontent.com/aaronhowellai/machine-learning-projects/main/efficient%20data%20loaders.png)

I will now implement a `Dataset()` class, which I will use to create a training and test dataset that I'll then use to create dataloaders.

In [ ]:
# create a toy dataset
X_train = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5,2.6],
    [2.3,-1.1],
    [2.7,-1.5]
])

y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([
    [-0.8,2.8],
    [2.6,-1.6],
])

y_test = torch.tensor([0,1])

In [ ]:
# defining a dataset class
class ToyDataset(Dataset):
  def __init__(self, X,y):
    self.features = X
    self.labels = y

  def __getitem__(self, index): # instructions for retrieving exactly one data record and the corresponding label
    one_x = self.features[index]
    one_y = self.labels[index]
    return one_x, one_y

  def __len__(self): # instructions for returning the total length of the dataset
    return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [ ]:
print(len(train_ds))

5


In [ ]:
# instantiating data loader class to sample from ToyDataset
torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0, # number of background processes
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [ ]:
# iterate over the training data loader
for idx, (x,y) in enumerate(train_loader):
  print(f"Batch {idx+1}:", x,y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


# **Typical Training Loop**

In [ ]:
# typical training loop
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2,num_outputs=2)
optimizer = torch.optim.SGD(
    model.parameters(), lr=.5
)

num_epochs = 3
for epoch in range(num_epochs):
  model.train()

  for batch_idx, (features, labels) in enumerate(train_loader):
    logits = model(features)
    loss = F.cross_entropy(logits, labels)

    optimizer.zero_grad() # reset the gradients to zero to prevent accumulation
    loss.backward()
    optimizer.step()

    ### logging
    print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
          f" | Batch {batch_idx+1:03d}/{len(train_loader):03d}"
          f" | Train Loss: {loss:.2f}")
print("\n")
model.eval()

Epoch: 001/003 | Batch 001/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 002/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 002/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 002/002 | Train Loss: 0.00




NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [ ]:
train_loader.batch_size

2

In [ ]:
# Total parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

# Trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Total parameters: 752
Trainable parameters: 752


In [ ]:
# make predictions using eval mode
model.eval()
with torch.no_grad():
  outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [ ]:
# class membership probabilities
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [ ]:
# view the class predictions
predictions = torch.argmax(outputs,dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [ ]:
# compare predictions to true labels
predictions == y_train

tensor([True, True, True, True, True])

In [ ]:
# compare predictions to true labels
len(predictions) == torch.sum(predictions == y_train)

tensor(True)

In [ ]:
# compare predictions to true labels
torch.sum(predictions == y_train)

tensor(5)

In [ ]:
# compute the prediction accuracy
def compute_accuracy(model, dataloader):
  model = model.eval()
  correct = 0.0
  total_examples = 0

  for idx, (features, labels) in enumerate(dataloader):

    with torch.no_grad():
      logits = model(features)

    predictions = torch.argmax(logits, dim=1)
    compare = labels == predictions
    correct += torch.sum(compare)
    total_examples += len(compare)

  return (correct / total_examples).item()

print(compute_accuracy(model, train_loader))

1.0


In [ ]:
# test set
print(compute_accuracy(model, test_loader))

1.0


In [ ]:
# save the model for reuse later
torch.save(model.state_dict(), "model.pth")

In [ ]:
# restore and load the model
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

# **Optimizing Training Performance with GPUs**
Training Deep Neural Networks on a Single GPU and Multiple GPUs

In [ ]:
# checking runtime supports gpu computing again
torch.cuda.is_available()

True

In [ ]:
# computations with gpu
tensor_1 = torch.tensor([1.,2.,3.])
tensor_2 = torch.tensor([4.,5.,6.])

tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")

print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')


## **Single-GPU training Example**



In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=.5)
num_epochs = 3

for epoch in range(num_epochs):
  model.train()
  for batch_idx, (features, labels) in enumerate(train_loader):
    features, labels = features.to(device), labels.to(device)
    logits = model(features)
    loss = F.cross_entropy(logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    ### logging
    print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
          f" | Batch {batch_idx+1:03d}/{len(train_loader):03d}"
          f" | Train/Val Loss: {loss:.2f}"
    )

Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 002/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 002/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 002/002 | Train/Val Loss: 0.00


## **Matrix Multiplication on GPU vs CPU**
Comparing speeds of matrix multiplication with tesnors of different magnitude.

In [ ]:
# compare with time module

# (3,2) 2D tensor
a_small = torch.tensor([[1,2],[3,4],[5,6]], dtype=torch.float32)
b_small = a_small.T

# gpu
a_small_gpu = a_small.to("cuda")
b_small_gpu = b_small.to("cuda")

# cpu
a_small_cpu = a_small.to("cpu")
b_small_cpu = b_small.to("cpu")

In [ ]:
%timeit a_small_gpu.matmul(b_small_gpu)

12.3 µs ± 1.41 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
%timeit a_small_cpu.matmul(b_small_cpu)

2.41 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### **Performance Improvment with GPU**

In [ ]:
a = torch.randn(1000, 1000)
b = a.T

In [ ]:
a.shape

torch.Size([1000, 1000])

**Numbers:**
* GPU: ~779 µs (microseconds)

* CPU: ~24 ms (milliseconds = 24,000 µs)

The computation is ~30x faster on GPU

In [ ]:
a_gpu = a.to("cuda")
b_gpu = b.to("cuda")
%timeit a_gpu.matmul(b_gpu)

779 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
a_cpu = a.to("cpu")
b_cpu = b.to("cpu")
%timeit a_cpu.matmul(b_cpu)

24 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Explanation:**
* GPU cores handle many operations in parallel.

* CPU handles fewer threads at a time — but with lower overhead.

* With larger tensors, data transfer time becomes negligible vs compute gain.